In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
train = pd.read_csv('C:/Users/dudwo/Desktop/ML/Data/titanic/train.csv')
test = pd.read_csv('C:/Users/dudwo/Desktop/ML/Data/titanic/test.csv')
train = train.drop(['Ticket','Cabin'],axis = 1) 
test = test.drop(['Ticket','Cabin'],axis = 1)   
train = train.fillna({"Embarked" : "S"})
embarked_mapping = {"S":1,"C":2,"Q":3}
train["Embarked"] = train["Embarked"].map(embarked_mapping)
test["Embarked"] = test["Embarked"].map(embarked_mapping)
combine = [train, test]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.',expand= False)
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(
        ['Lady', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
    dataset['Title'] = dataset['Title'].replace('Mile', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
title_mapping = {"Mr":1,"Miss":2,"Mrs":3,"Master":4,"Royal":5,"Rare":6}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
train = train.drop(['Name','PassengerId'],axis =1)
test = test.drop(['Name','PassengerId'],axis =1)
combine = [train,test]
sex_mapping = {"male":0,"female":1}
for dataset in combine :
    dataset['Sex'] = dataset["Sex"].map(sex_mapping)
train['Age'] = train['Age'].fillna(-0.5)
test['Age'] = test['Age'].fillna(-0.5)
bins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
labels = ['Unknown','Baby','Child','Teenager','Student','Young Adult','Adult','Senior']
train['AgeGroup'] = pd.cut(train["Age"],bins, labels= labels)
test['AgeGroup'] = pd.cut(test["Age"],bins, labels=labels)
age_title_mapping = {1:"Young Adult",2:"Student",3:"Adult",4:"Baby",5:"Adult",6:"Adult",}
for x in range(len(train["AgeGroup"])):
    if train["AgeGroup"][x] == "Unknown" :
        train["AgeGroup"][x] = age_title_mapping[train["Title"][x]]
for x in range(len(test["AgeGroup"])):
    if test["AgeGroup"][x] == "Unknown" :
        test["AgeGroup"][x] = age_title_mapping[test["Title"][x]]
age_mappaing = {"Baby":1,"Child":2,"Teenager":3,"Student":4,"Young Adult":5,"Adult":6,"Senior":7}
train['AgeGroup'] = train['AgeGroup'].map(age_mappaing)
test['AgeGroup'] = test['AgeGroup'].map(age_mappaing)
train = train.drop(['Age'],axis =1)
test = test.drop(['Age'],axis =1)
train['FareBand'] = pd.qcut(train['Fare'], 4, labels= [1,2,3,4])
test['FareBand'] = pd.qcut(train['Fare'],4, labels=[1,2,3,4])
train = train.drop(['Fare'],axis = 1)
test = test.drop(['Fare'],axis=1)

In [6]:
train_data = train.drop('Survived', axis = 1)
target = train['Survived']
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Pclass      891 non-null int64
Sex         891 non-null int64
SibSp       891 non-null int64
Parch       891 non-null int64
Embarked    891 non-null int64
Title       891 non-null float64
AgeGroup    891 non-null int64
FareBand    891 non-null category
dtypes: category(1), float64(1), int64(6)
memory usage: 49.9 KB


In [8]:
# Cross_Validation 실행

from sklearn.model_selection import KFold
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [11]:
# RandomForestClassifier 실행

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=13)
print(clf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=13, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [13]:
# Cross_val_score 점수
# f1 Scoring

from sklearn.model_selection import cross_val_score
scoring = 'f1'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
print(score)

[0.8        0.72413793 0.66666667 0.71428571 0.72727273 0.73333333
 0.79452055 0.73015873 0.72972973 0.76315789]


In [14]:
# accuracy Scoring

from sklearn.model_selection import cross_val_score
scoring = 'accuracy'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
print(score)

[0.82222222 0.86516854 0.78651685 0.78651685 0.86516854 0.82022472
 0.82022472 0.84269663 0.79775281 0.80898876]


In [15]:
# recall Scoring

from sklearn.model_selection import cross_val_score
scoring = 'recall'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
print(score)

[0.82051282 0.66666667 0.67741935 0.69444444 0.7037037  0.70967742
 0.80555556 0.70967742 0.71794872 0.73809524]


In [16]:
# precision Scoring

from sklearn.model_selection import cross_val_score
scoring = 'precision'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
print(score)

[0.74358974 0.77777778 0.67857143 0.77419355 0.75       0.79310345
 0.78378378 0.77419355 0.875      0.86111111]
